Taken from chapter 5 of [Machine learning for cyberscurity cookbook](https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/blob/master/Chapter05/CAPTCHA%20Breaker/Processing%20a%20CAPTCHA%20Dataset.ipynb)

## Processing a Captcha dataset

In [2]:
import os

captcha_images_folder = "../input/captcha-images/captcha_images"
captchas = [
    os.path.join(captcha_images_folder, f) for f in os.listdir(captcha_images_folder)
]

In [3]:
import cv2

def preprocess_CAPTCHA(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_with_border = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    preprocessed = cv2.threshold(
        gray_with_border, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )[1]
    return gray_with_border, preprocessed

In [4]:
def get_CAPTCHA_label(path_to_file):
    filename = os.path.basename(path_to_file)
    label = filename.split(".")[0]
    return label

In [7]:
def find_bounding_rectangles_of_contours(contours):
    letter_bounding_rectangles= []
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if w/h > 1.25:
            half_width = int(w/2)
            letter_bounding_rectangles.append((x,y,half_width, h))
            letter_bounding_rectangles.append((x+half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append((x,y,w,h))
    return letter_bounding_rectangles

In [5]:
def CAPTCHA_to_gray_scale_and_bounding_rectangles(captcha_image_file):
    image = cv2.imread(captcha_image_file)
    gray, preprocessed = preprocess_CAPTCHA(image)
    contours = cv2.findContours(
        preprocessed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    contours = contours[0]
    letter_bounding_rectangles = find_bounding_rectangles_of_contours(contours)
    letter_bounding_rectangles = sorted(letter_bounding_rectangles, key=lambda x: x[0])
    return gray, letter_bounding_rectangles

In [6]:
def bounding_rectangle_to_letter_image(letter_bounding_box, grayscaled):
    x,y,w,h = letter_bounding_box
    letter_image = grayscaled[y-2:y+h+2, x-2:x+w+2]
    return letter_image

In [9]:
captcha_processing_output_folder = "extracted_letter_images"
character_counts = {}


def crop_bounding_rectangles_and_save_to_file(
    letter_bounding_rectangles, gray, captcha_label
):
    """Saves the individual letters of a CAPTCHA."""
    for letter_bounding_rectangle, current_letter in zip(
        letter_bounding_rectangles, captcha_label
    ):
        letter_image = bounding_rectangle_to_letter_image(
            letter_bounding_rectangle, gray
        )

        save_path = os.path.join(captcha_processing_output_folder, current_letter)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        character_count = character_counts.get(current_letter, 1)

        p = os.path.join(save_path, str(character_count) + ".png")
        cv2.imwrite(p, letter_image)

        character_counts[current_letter] = character_count + 1

In [11]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=fe06ebfb81baac891e098122622e0076e2f680c1a315f785a9d003e4b0309f59
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [15]:
import imutils
import numpy as np

for captcha_image_file in captchas:
    captcha_label = get_CAPTCHA_label(captcha_image_file)
    gray, letter_bounding_rectangles = CAPTCHA_to_gray_scale_and_bounding_rectangles(captcha_image_file)
    if( len(letter_bounding_rectangles) != 4):
        continue
    crop_bounding_rectangles_and_save_to_file(letter_bounding_rectangles,gray,captcha_label)
    

This is an example of a segmented letter:

![](./extracted_letter_images/L/339.png)

In [ ]:
## Next up Training the actual captcha solver
